In [3]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [4]:
var('xx')
Q.<ii> = NumberField(xx^2 + 1)

In [5]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [6]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 10.2 s, sys: 374 ms, total: 10.5 s
Wall time: 26.3 s


576

In [7]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 5.35 s, sys: 230 ms, total: 5.58 s
Wall time: 29.7 s


2

In [8]:
with open('all_L_sets.pickle', 'rb') as fil:
    all_L_sets = pickle.load(fil)

In [9]:
L_set_base = ('E1', 'G4', 'E2', 'G3', 'E3')
possible_L_sets = [L_set for L_set in all_L_sets if L_set[0:3] == ('E1', 'G4', 'E3')]

In [10]:
%time L_set_ideals, all_ideals = SE1.find_conditions_on_L_sets(possible_L_sets, "eck")

CPU times: user 43.5 s, sys: 5.45 ms, total: 43.5 s
Wall time: 43.5 s


In [11]:
L_set_ideals

[[('E1', 'G4', 'E3', 'G6', 'E6'), []],
 [('E1', 'G4', 'E3', 'G6', 'F45'), []],
 [('E1', 'G4', 'E3', 'G6', 'F24'),
  [Ideal (4*c + (ii)*d + (-ii)*e + (ii + 2)*f, 5*d^2 + (4*ii - 2)*d*e + (-4*ii - 3)*e^2 + (-8*ii - 6)*d*f + (-36*ii + 38)*e*f + (24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1,
   Ideal (4*c + (-ii)*d + (ii)*e + (-ii + 2)*f, 5*d^2 + (-4*ii - 2)*d*e + (4*ii - 3)*e^2 + (8*ii - 6)*d*f + (36*ii + 38)*e*f + (-24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1]],
 [('E1', 'G4', 'E3', 'F13', 'E6'), []],
 [('E1', 'G4', 'E3', 'F13', 'E5'), []],
 [('E1', 'G4', 'E3', 'F13', 'E2'), []],
 [('E1', 'G4', 'E3', 'G5', 'F46'), []],
 [('E1', 'G4', 'E3', 'G5', 'E5'), []],
 [('E1', 'G4', 'E3', 'G5', 'F24'),
  [Ideal (4*c + (ii)*d + (ii + 2)*e + (-ii)*f, 5*d^2 + (-8*ii - 6)*d*e + (24*ii - 7)*e^2 + (4*ii - 2)*d*f + 

In [13]:
ideales = [_ for i in range(len(all_ideals))]
poly_es = [_ for i in range(len(all_ideals))]
e_sosts = [_ for i in range(len(all_ideals))]
poly_ds = [_ for i in range(len(all_ideals))]
d_sosts = [_ for i in range(len(all_ideals))]
for i in range(len(all_ideals)):
    ideales[i] = all_ideals[i]
    poly_es[i] = ideales[i].gens()[0]
    e_sosts[i] = {e: -(poly_es[i]-poly_es[i].coefficient(e)*e)/poly_es[i].coefficient(e)}
    poly_ds[i] = [gen for gen in ideales[i].subs(e_sosts[i]).gens() if gen !=0][0]
    d_sosts[i] = {d:-(poly_ds[i]-poly_ds[i].coefficient(d)*d)/poly_ds[i].coefficient(d)}
    print([gen.subs(e_sosts[i]).subs(d_sosts[i]) for gen in ideales[i].gens()])

[0, 0]
[0, 0]
[0, 0]
[0, 0]


In [14]:
SE1_ = [SE1.subs(e_sosts[0]).subs(d_sosts[0])]+[SE1.subs(e_sosts[i]).subs(d_sosts[i]).subs({c:l, f:m}) for i in range(1, len(all_ideals))]

In [15]:
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
current_conds = list(set(matrix([[SE1_[0].coefficient(mn) for mn in mon], [SE1_[2].coefficient(mn) for mn in mon]]).minors(2)))
prim_dec = P.ideal(current_conds).radical().primary_decomposition()
prim_dec

[Ideal (5*c*l + (4*ii + 3)*f*l + (-2*ii + 1)*c*m + (2*ii - 1)*f*m) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1]

In [16]:
poly = prim_dec[0].gens()[0]
poly.coefficient(l), poly.coefficient(m)

(5*c + (4*ii + 3)*f, (-2*ii + 1)*c + (2*ii - 1)*f)

In [17]:
sost1 = {l: poly.coefficient(m), m: -poly.coefficient(l)}
SE1_[0].are_cubics_same(SE1_[2].subs(sost1))

True

In [18]:
%%time 
L_sets = list(set([get_permuted_L_set(perm) for perm in SE1_[1].find_admissible_permutations()]))
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
for L_set in L_sets:
    proj = SE1_[1].find_projectivity(SE1_[1].L_set_base, L_set)
    eqn = SE1_[1].eqn.subs(change_coordinates(proj))
    eqn = list(eqn.factor())[-1][0]
    current_conds = list(set(matrix([[SE1_[0].coefficient(mn) for mn in mon], [eqn.coefficient(mn) for mn in mon]]).minors(2)))
    prim_dec = P.ideal(current_conds).radical().primary_decomposition()
    if [(L_set,ideal) for ideal in prim_dec if len(ideal.gens())==1] != []:
        L_set_ideal = (L_set, prim_dec[0])
        break
L_set_ideal

CPU times: user 7.47 s, sys: 31.1 ms, total: 7.5 s
Wall time: 7.54 s


(('E1', 'F14', 'F25', 'F16', 'F56'),
 Ideal (3*c*l - f*l - c*m - f*m) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1)

In [19]:
poly = L_set_ideal[1].gens()[0]
poly

3*c*l - f*l - c*m - f*m

In [20]:
sost = {l: poly.coefficient(m), m: -poly.coefficient(l)}
SE1_[0].are_cubics_same(SE1_[1].eqn.subs(change_coordinates(proj)).subs(sost))

True

In [21]:
SE1_1 = SE1_[0]

In [22]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 3.13 s, sys: 189 ms, total: 3.32 s
Wall time: 25.8 s


576

In [23]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 1.85 s, sys: 193 ms, total: 2.05 s
Wall time: 9.17 s


4

In [24]:
lines_perms = []
for simm in simm_SE1_1:
    lines_perms.append(Permutation(SE1_1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(4, 'C4')

In [25]:
lines_perms_group.gens_small()

[(2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20)]

In [26]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46')]

In [27]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens() if perm.order()==4]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46'),
 ('E1', 'G4', 'F34', 'F16', 'E2', 'E6')]

In [28]:
lines_perms_group.gens()

[(),
 (2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20),
 (2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
 (2,22,19,3)(4,23,7,21)(5,6,25,26)(8,11,13,16)(9,17,14,12)(18,20,27,24)]